### Imports

In [1]:
import org.apache.spark.sql.SparkSession

Intitializing Scala interpreter ...

Spark Web UI available at http://Varun-CK:4040
SparkContext available as 'sc' (version = 2.3.0, master = local[*], app id = local-1579464674414)
SparkSession available as 'spark'


import org.apache.spark.sql.SparkSession


### Creating Spark Session

In [2]:
val spark = SparkSession.builder.getOrCreate()

2020-01-20 01:41:26 WARN  SparkContext:66 - Using an existing SparkContext; some configuration may not take effect.


spark: org.apache.spark.sql.SparkSession = org.apache.spark.sql.SparkSession@34ac69cd


### Data

In [4]:
val tagsDF = spark
    .read
    .option("multiLine", true)
    .option("inferSchema", true)
    .json("..\\Resources\\tags_sample.json")

tagsDF: org.apache.spark.sql.DataFrame = [stackoverflow: array<struct<tag:struct<author:string,frameworks:array<struct<id:bigint,name:string>>,id:bigint,name:string>>>]


### Explode

In [5]:
val df = tagsDF
    .select(explode($"stackoverflow") as "stackoverflow_tags")
    .select(
      $"stackoverflow_tags.tag.id" as "id",
      $"stackoverflow_tags.tag.author" as "author",
      $"stackoverflow_tags.tag.name" as "tag_name",
      $"stackoverflow_tags.tag.frameworks.id" as "frameworks_id",
      $"stackoverflow_tags.tag.frameworks.name" as "frameworks_name"
    )

df: org.apache.spark.sql.DataFrame = [id: bigint, author: string ... 3 more fields]


In [7]:
df.show(false)

+---+--------------+--------+-------------+--------------------------------+
|id |author        |tag_name|frameworks_id|frameworks_name                 |
+---+--------------+--------+-------------+--------------------------------+
|1  |Martin Odersky|scala   |[1, 2]       |[Play Framework, Akka Framework]|
|2  |James Gosling |java    |[1, 2]       |[Apache Tomcat, Spring Boot]    |
+---+--------------+--------+-------------+--------------------------------+



### array_contains

In [8]:
import org.apache.spark.sql.functions.array_contains

import org.apache.spark.sql.functions.array_contains


In [9]:
df.select("*")
  .where(array_contains($"frameworks_name","Play Framework"))
  .show(false)

+---+--------------+--------+-------------+--------------------------------+
|id |author        |tag_name|frameworks_id|frameworks_name                 |
+---+--------------+--------+-------------+--------------------------------+
|1  |Martin Odersky|scala   |[1, 2]       |[Play Framework, Akka Framework]|
+---+--------------+--------+-------------+--------------------------------+



### Closing Spark Session

In [10]:
spark.stop()